In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import pandas as pd
import numpy as np
from efficientnet_pytorch import *
from fastai import *
from fastai.vision import *
from fastai.callbacks import SaveModelCallback,MixUpCallback
from sklearn.metrics import roc_auc_score,f1_score,recall_score
from sklearn.model_selection import train_test_split,StratifiedKFold
from pathlib import Path
import gc

In [ ]:
path = Path('/kaggle/input/melanoma-external-malignant-256')
path128 = Path('/kaggle/input/siimisic-melanoma-classification-128')

In [ ]:
test = pd.read_csv('../input/siimisic-melanoma-classification-128/Melonoma_cancer_tabular/Melonoma_cancer/test_final_stat.csv')

In [ ]:
train = pd.read_csv(path/'train_concat.csv')

In [ ]:
train.head()

In [ ]:
seed=101
folds=5

train['kfold']=-1
train = train.sample(frac=1.,random_state=seed).reset_index(drop=True)
y = train.target.values
kf = StratifiedKFold(n_splits=folds,shuffle=True,random_state = seed)
for f,(t_,v_) in enumerate(kf.split(X=train,y=y)):
    train.loc[v_,'kfold'] = f

In [ ]:
test_img = (ImageList.from_df(test,path=path/'test',folder='test',suffix='.jpg',cols='image_name'))

In [ ]:
model_name = 'efficientnet-b3'

def get_model(model_name,n_cls):
    model = EfficientNet.from_pretrained(model_name, num_classes=n_cls)
    return model

In [ ]:
def data_fold(fold):
    
    train_idx,valid_idx = train[train.kfold!=fold].index,train[train.kfold==fold].index
    data = (ImageList.from_df(train,path=path/'train',folder='train',suffix='.jpg',cols='image_name')
                     .split_by_idxs(train_idx,valid_idx)
                     .label_from_df(cols='target') 
                     .transform(get_transforms(do_flip=True,max_zoom=1.2,flip_vert=True),size=224)
                     .add_test(test_img)
                     .databunch(bs=64)).normalize(imagenet_stats)
    
    return data
    

In [ ]:
test_sc = {}

for i in range(0,folds,2):
    
    data = data_fold(i)
    model_name = str('fold_'+str(i)+'__best_model')
    arch = models.resnet50
    learn = cnn_learner(data , arch , metrics =[AUROC()] , model_dir = '/kaggle/working').mixup().to_fp16()  
    
    learn.fit_one_cycle(20, slice(1e-1/2), callbacks = [SaveModelCallback(learn, every ='improvement', monitor ='auroc', name = model_name)])    
    preds,_ = learn.get_preds(DatasetType.Test)
    preds = preds.numpy()[:,1]
    name = 'fold__'+str(i)
    test_sc[name] = preds

    data,learn = None,None
    gc.collect()

In [ ]:
test_sc

In [ ]:
test_sc = np.column_stack((test_sc['fold__0'],
                           test_sc['fold__2'],                         
                           test_sc['fold__4']                          
                          ))


In [ ]:
sub = pd.DataFrame({ 'image_name' : test.image_name.values ,'target' : test_sc.mean(axis=1) })
sub.to_csv('submission.csv',index=False)